## E15 - 번역기를 만들어보자

기계 번역에 대해   
규칙 기반의 기계 번역 -> 통계적 기계 번역 -> 신경망 기계 번역(2016년, 구글 번역기)   

RNN   
1. one-to-one: 일반적인 피드 포워드 신경망   
2. one-to-many: 하나의 입력으로부터 다수의 출력, image captioning   
3. many-to-one: 여러 입력으로부터 하나의 결과 출력, 텍스트 분류, 스펨 메일 분류기    
4. many-to-many: 5번과 달리 어떤 스텝에서 출력이 시작, 기계 번역기, 문장을 모두 읽은 후에야 번역이 가능    
5. many-to-many: 매 스텝마다 바로 출력, 입력 문장의 각 단어가 무엇인지, 개체명 인식, 품사 태깅에 사용   

seq2seq 구조(번역기의 기본 구조)   
두 개의 RNN을 연결한 구조, 입력 문장을 받는 RNN(인코더), 그 다음 RNN(디코더)   
인코더에서 어떤 데이터 X에서 저차원의 feature vector z를 만들어 낸다.    
디코더는 저차원의 feature vector z로부터 정보를 복원하여 다시 어떤 데이터 X'을 재생성한다.   

인코더 RNN은 입력문장을 해석해서 만든 hidden state 벡터.   
A언어의 문장 X를 z라는 hidden state로 해석한 후 z를 다시 B언어의 문장 Y로 재생성하는 구조.
디코더 RNN은 인코더 RNN의 마지막 스텝의 hidden state를 전달받는다.    
이 hidden state를 토대로 출력 문장을 만들어 낸다.   
_GO: 출력 시작을 의미하는 단어, EOS: 출력 종료 단어   

조건적 언어 모델(conditional language model): 조건에 맞는 문장을 생성하라. 기계 번역   
X라는 영어 문장을 Y라는 프랑스어 문장을 만들기, 프랑스어라는 조건을 붙여준다.   
즉, X -> c -> Y, RNN-RNN-RNN -> seq2seq   

교사 강요(teacher forcing)   
seq2seq는 훈련 과정과 테스트 과정이 있는데 테스트 과정에서 문장을 만들어낸다.   
훈련 과정에서 잘못된 hidden state를 넘겨준다면 테스트 과정에서 잘못된 문장을 만들어 낼 것이다.
훈련 과정에서는 정답을 알고 있는 상황이므로 이전 time step의 예측값이 아닌 실제값을 입력으로 사용할 수 있다.   
이를 교사 강요라 하며 실제 sequence 데이터의 생성 모델에서 사용하는 기법이다.   
훈련 데이터 이외의 결과를 생성해내는 능력을 기르는 데에는 방해가 될 수 있다.   

seq2seq는 단어 수준 또는 문자 수준으로 입출력 단위를 정할 수 있다.   
문자 수준의 출력층은 알파펫의 경우 대소문자 구분하지 않고 특수문자를 포함하더라도 100을 넘지 않는다.   
어떤 수준이 번역에 유리할까? 둘은 trade-off관계다.
단어 수준의 번역은 단어의 변화가 셀 수 없을 정도로 많고 띄어쓰기가 있을 경우 전처리가 어렵다.   
문자 수준의 번역은 너무 작게 쪼개져서 단어 안에 내재된 정보가 소실된다. 단어를 이루는 패턴을 학습해야 한다.   
최신 자연어 처리 흐름은 subword기반의 번역이 주를 이룬다.

 사용할 데이터: https://www.manythings.org/anki/   
 
 mkdir -p ~/aiffel/translator_seq2seq/data   
 mkdir -p ~/aiffel/translator_seq2seq/models   
 wget https://www.manythings.org/anki/fra-eng.zip   
 mv fra-eng.zip  ~/aiffel/translator_seq2seq/data   
 cd ~/aiffel/translator_seq2seq/data && unzip fra-eng.zip   

In [1]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np

In [2]:
import os
file_path = os.getenv('HOME')+'/aiffel/translator_seq2seq/data/fra.txt'
lines = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')
print('전체 샘플의 수 :',len(lines))
lines.sample(5) #샘플 5개 출력

전체 샘플의 수 : 178009


,eng,fra,cc
58739,We don't have any proof.,Nous ne disposons d'aucune preuve.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
88182,What will the neighbors say?,Que diront les voisins ?,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
91472,I read the story three times.,J'ai lu l'histoire trois fois.,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
71877,"She has about 2,000 books.",Elle a à peu près 2000 livres.,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
170552,Tom doesn't know the difference between linen ...,Tom ne fait pas la différence entre le lin et ...,CC-BY 2.0 (France) Attribution: tatoeba.org #6...


In [3]:
lines = lines[['eng', 'fra']][:50000] # 5만개 샘플 사용, 세 번째 열은 제외
lines.sample(5)

,eng,fra
22603,You won't be shot.,Vous ne serez pas fusillés.
11002,I hate that guy.,Je déteste ce mec.
9285,They were dead.,Elles étaient mortes.
11468,I wish you luck.,Je vous souhaite bonne chance.
12997,We must succeed.,Il nous faut réussir.


In [4]:
# 시작 토큰과 종료 토큰 추가
sos_token = '\t'
eos_token = '\n'
lines.fra = lines.fra.apply(lambda x : '\t '+ x + ' \n')
print('전체 샘플의 수 :',len(lines))
lines.sample(5)

전체 샘플의 수 : 50000


,eng,fra
31062,Let me pay my share.,\t Laisse-moi payer ma part. \n
4619,We must obey.,\t Il nous faut obéir. \n
28287,Are you ready to go?,\t Êtes-vous prête à partir ? \n
8403,I'll cancel it.,\t Je l'annulerai. \n
8098,I like almonds.,\t J'aime les amandes. \n


In [5]:
# 단어장을 만들기, 영어와 프랑스어별로

eng_tokenizer = Tokenizer(char_level=True)   # 문자 단위로 Tokenizer를 생성합니다. 
eng_tokenizer.fit_on_texts(lines.eng)               # 50000개의 행을 가진 eng의 각 행에 토큰화를 수행
input_text = eng_tokenizer.texts_to_sequences(lines.eng)    # 단어를 숫자값 인덱스로 변환하여 저장
input_text[:3]

[[19, 3, 8], [10, 5, 8], [10, 5, 8]]

In [6]:
fra_tokenizer = Tokenizer(char_level=True)   # 문자 단위로 Tokenizer를 생성합니다. 
fra_tokenizer.fit_on_texts(lines.fra)                 # 50000개의 행을 가진 fra의 각 행에 토큰화를 수행
target_text = fra_tokenizer.texts_to_sequences(lines.fra)     # 단어를 숫자값 인덱스로 변환하여 저장
target_text[:3]

[[11, 1, 19, 4, 1, 33, 1, 12],
 [11, 1, 3, 4, 13, 7, 5, 1, 33, 1, 12],
 [11, 1, 3, 4, 13, 7, 5, 14, 1, 12]]

In [7]:
eng_vocab_size = len(eng_tokenizer.word_index) + 1
fra_vocab_size = len(fra_tokenizer.word_index) + 1
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)

영어 단어장의 크기 : 51
프랑스어 단어장의 크기 : 73


In [8]:
max_eng_seq_len = max([len(line) for line in input_text])
max_fra_seq_len = max([len(line) for line in target_text])
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)
# 패딩처리하기 위해

영어 시퀀스의 최대 길이 23
프랑스어 시퀀스의 최대 길이 76


In [9]:
print('전체 샘플의 수 :',len(lines))
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

전체 샘플의 수 : 50000
영어 단어장의 크기 : 51
프랑스어 단어장의 크기 : 73
영어 시퀀스의 최대 길이 23
프랑스어 시퀀스의 최대 길이 76


In [11]:
# 인코더인 영어 시퀀스와 달리 프랑스어 시퀸스는 디코더 출려과 비교할 정답 데이터로서 하나와
# 교사 강요을 위해 디코더 입력으로서 하나로 두 개를 만든다.
# 입력 시퀀스는 <eos>토큰은 필요 없고 비교할 시퀀스는 <sos>토큰이 필요 없다.

encoder_input = input_text
# 종료 토큰 제거
decoder_input = [[ char for char in line if char != fra_tokenizer.word_index[eos_token] ] for line in target_text] 
# 시작 토큰 제거
decoder_target = [[ char for char in line if char != fra_tokenizer.word_index[sos_token] ] for line in target_text]

In [12]:
print(decoder_input[:3])
print(decoder_target[:3])

# 12: <eos>토큰, 11: <sos>토큰

[[11, 1, 19, 4, 1, 33, 1], [11, 1, 3, 4, 13, 7, 5, 1, 33, 1], [11, 1, 3, 4, 13, 7, 5, 14, 1]]
[[1, 19, 4, 1, 33, 1, 12], [1, 3, 4, 13, 7, 5, 1, 33, 1, 12], [1, 3, 4, 13, 7, 5, 14, 1, 12]]


In [13]:
# 패딩 처리하기
encoder_input = pad_sequences(encoder_input, maxlen = max_eng_seq_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen = max_fra_seq_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen = max_fra_seq_len, padding='post')
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (50000, 23)
프랑스어 입력데이터의 크기(shape) : (50000, 76)
프랑스어 출력데이터의 크기(shape) : (50000, 76)


In [14]:
print(encoder_input[0])

[19  3  8  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]


In [15]:
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (50000, 23, 51)
프랑스어 입력데이터의 크기(shape) : (50000, 76, 73)
프랑스어 출력데이터의 크기(shape) : (50000, 76, 73)


In [16]:
print(encoder_input[0])

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [17]:
# 학습데이터와 검증데이터 분리하기

n_of_val = 3000

encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

print('영어 학습데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 학습 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 학습 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 학습데이터의 크기(shape) : (50000, 23, 51)
프랑스어 학습 입력데이터의 크기(shape) : (50000, 76, 73)
프랑스어 학습 출력데이터의 크기(shape) : (50000, 76, 73)


In [18]:
# 모델 훈련하기

from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model

# 인코더 설계
# 입력 텐서 생성, 입력 문장을 저장할 변수 텐서
encoder_inputs = Input(shape=(None, eng_vocab_size))
# hidden size가 256인 인코더의 LSTM 셀 생성
encoder_lstm = LSTM(units = 256, return_state = True)
# 디코더로 전달할 hidden state, cell state를 리턴. encoder_outputs은 여기서는 불필요.
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
# hidden state와 cell state를 다음 time step으로 전달하기 위해서 별도 저장.
encoder_states = [state_h, state_c]

# 디코더 설계
# 입력 텐서 생성.
decoder_inputs = Input(shape=(None, fra_vocab_size))
# hidden size가 256인 인코더의 LSTM 셀 생성
decoder_lstm = LSTM(units = 256, return_sequences = True, return_state=True)
# decoder_outputs는 모든 time step의 hidden state
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state = encoder_states)

decoder_softmax_layer = Dense(fra_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

In [19]:
# 모델 정의
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 51)]   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 73)]   0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 256), (None, 315392      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 256),  337920      input_2[0][0]                    
                                                                 lstm[0][1]                   

In [20]:
model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train, \
          validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size=128, epochs=50)

Epoch 1/50
368/368 [==============================] - 6s 17ms/step - loss: 0.9450 - val_loss: 0.8151
Epoch 2/50
368/368 [==============================] - 6s 16ms/step - loss: 0.5745 - val_loss: 0.6531
Epoch 3/50
368/368 [==============================] - 6s 16ms/step - loss: 0.4737 - val_loss: 0.5656
Epoch 4/50
368/368 [==============================] - 6s 16ms/step - loss: 0.4138 - val_loss: 0.5167
Epoch 5/50
368/368 [==============================] - 6s 16ms/step - loss: 0.3750 - val_loss: 0.4816
Epoch 6/50
368/368 [==============================] - 6s 16ms/step - loss: 0.3471 - val_loss: 0.4561
Epoch 7/50
368/368 [==============================] - 6s 16ms/step - loss: 0.3255 - val_loss: 0.4366
Epoch 8/50
368/368 [==============================] - 6s 16ms/step - loss: 0.3087 - val_loss: 0.4256
Epoch 9/50
368/368 [==============================] - 6s 16ms/step - loss: 0.2948 - val_loss: 0.4115
Epoch 10/50
368/368 [==============================] - 6s 16ms/step - loss: 0.2833 - val_lo

In [21]:
# 테스트하기
# 훈련과 달리 디코더에서 토큰이 '\n'이 나올 때까지 예측하면서 반복한다.

# 인코더 정의
encoder_model = Model(inputs = encoder_inputs, outputs = encoder_states)
encoder_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, 51)]        0         
_________________________________________________________________
lstm (LSTM)                  [(None, 256), (None, 256) 315392    
Total params: 315,392
Trainable params: 315,392
Non-trainable params: 0
_________________________________________________________________


In [22]:
# 디코더 정의

# 이전 time step의 hidden state를 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
# 이전 time step의 cell state를 저장하는 텐서
decoder_state_input_c = Input(shape=(256,))
# 이전 time step의 hidden state와 cell state를 하나의 변수에 저장
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# decoder_states_inputs를 현재 time step의 초기 상태로 사용.
# 구체적인 동작 자체는 def decode_sequence()에 구현.
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state = decoder_states_inputs)
# 현재 time step의 hidden state와 cell state를 하나의 변수에 저장.
decoder_states = [state_h, state_c]

In [23]:
# 출력층
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)
decoder_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None, 73)]   0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 256),  337920      input_2[0][0]                    
                                                                 input_3[0][0]              

In [24]:
# 정수->단어, 단어->정수

eng2idx = eng_tokenizer.word_index
fra2idx = fra_tokenizer.word_index
idx2eng = eng_tokenizer.index_word
idx2fra = fra_tokenizer.index_word

In [25]:
# decode_sequence(): 입력 인자는 번역하고자 하는 문장의 정수 시퀀스

def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, fra_vocab_size))
    target_seq[0, 0, fra2idx['\t']] = 1.

    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = idx2fra[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_fra_seq_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1, fra_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [26]:
import numpy as np
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스 (자유롭게 선택해 보세요)
    input_seq = encoder_input[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', lines.eng[seq_index])
    print('정답 문장:', lines.fra[seq_index][1:len(lines.fra[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-1]) # '\n'을 빼고 출력

-----------------------------------
입력 문장: Run!
정답 문장:  Cours ! 
번역기가 번역한 문장:  couriez ! 
-----------------------------------
입력 문장: I left.
정답 문장:  Je suis partie. 
번역기가 번역한 문장:  je suis partie. 
-----------------------------------
입력 문장: Call us.
정답 문장:  Appelez-nous ! 
번역기가 번역한 문장:  appelez ! 
-----------------------------------
입력 문장: How nice!
정답 문장:  Comme c'est gentil ! 
번역기가 번역한 문장:  comme c'est chaud ! 
-----------------------------------
입력 문장: Turn left.
정답 문장:  Tourne à gauche. 
번역기가 번역한 문장:  fieus-le à cela. 
